In [6]:
import numpy as np
import itertools as it

def naive(A,B,pi,obs,print_steps=True):
    """
    B should be column stochastic
    obs contains the indices of the thing
    """
    T = len(obs)
    N = A.shape[0]
    M = B.shape[0]
    
    tot = 0
    for seq in it.product(range(N),repeat=T):
        # seq is a sequence of length T
        
        prob = pi[seq[0]]
        for i in xrange(T-1):
            # Multiply by the probability of moving to the state in seq[i+1],
            # given that you are in the state in seq[i]
            prob *= A[seq[i],seq[i+1]]
            # Get the probability of the i-th observation draw, given that 
            # you are in the state seq[i]
            prob *= B[obs[i],seq[i]]
        # Do the last beta thing
        prob *= B[obs[T-1],seq[T-1]]
        if print_steps:
            print "Probability of ", seq, ":", prob
        tot += prob
    if print_steps:
        print "Total probability for ", obs, ": ", tot
    return tot

def alpha_pass(A,B,pi,obs,print_out=True):
    T = len(obs) # of observations
    N = A.shape[0] # of states
    M = B.shape[0]
    
    alpha = np.zeros((N,T))
    alpha[:,0] = pi*B[obs[0],:]
    
    for i in xrange(1,T):
        alpha[:,i] = np.dot(A.T,alpha[:,i-1])*B[obs[i],:]
    c = 1/np.sum(alpha,axis=0)
    alpha = c*alpha
    if print_out:
        print "Alpha:\n",alpha
    return alpha, c

def prob1():
    print "Problem 1:"
    print "Part a:"
    A = np.array([[0.7,0.3],[0.4,0.6]])
    B = np.array([[0.1,0.7],[0.4,0.2],[0.5,0.1]])
    pi = np.array([0.0,1.0])
    obs = np.array([1.0,0.0,2.0])
    naive(A,B,pi,obs)
    
    print "\nPart b:"
    alpha_pass(A,B,pi,obs) 
    
    print "\nPart c:"
    print "I implemented the direct computation of P(O|lambda) and the alpha pass as algorithms, exactly like it said, and they worked."
    
    print "\nPart d:"
    print "For part (a), we have five multiplications for each of the eight possible sequences, so the work factor is 40"
    print "In terms of N and T, we have 2^N possible sequences with 2T-1 multiplications each, so work factor is 2^N*(2T-1)"
    print "For part (b), we have fourteen multiplications, so the work factor is 14."
    print "In terms of N and T, we have N*(N+1) multiplications per step, except the first step is only N, so total of N+(T-1)*N*(N+1)"
    print "We did not account for normalization in any of these calculations."
    
def prob2():
    print "\nProblem 2:"
    print "The sequence (1,1,0) (CCH) has the highest probability of producing the observation sequence, so it is the best in the dynamic programming sense."
    print "The sequence (1,1,0) (CCH) also has the highest probability of being correct at each step, according to the alpha pass, so it is also the best in the HMM sense."
 
def prob3():
    print "\nProblem 3:"
    tot = 0
    A = np.array([[0.7,0.3],[0.4,0.6]])
    B = np.array([[0.1,0.7],[0.4,0.2],[0.5,0.1]])
    pi = np.array([0.6,0.4])
    alpha = np.zeros((2,4))
    for obs in it.product(range(3),repeat=4):
        prob = naive(A,B,pi,obs,print_steps=False)
        tot += prob
        to_add, cT = alpha_pass(A,B,pi,obs,print_out=False)
        alpha += to_add
    print "Total probability over all observations: ", tot
    print "Using forward algorithm:\n", alpha, np.sum(alpha[:,-1])
    
prob1()
prob2()
prob3()

Problem 1:
Part a:
Probability of  (0, 0, 0) : 0.0
Probability of  (0, 0, 1) : 0.0
Probability of  (0, 1, 0) : 0.0
Probability of  (0, 1, 1) : 0.0
Probability of  (1, 0, 0) : 0.0028
Probability of  (1, 0, 1) : 0.00024
Probability of  (1, 1, 0) : 0.0168
Probability of  (1, 1, 1) : 0.00504
Total probability for  [ 1.  0.  2.] :  0.02488

Part b:
Alpha:
[[ 0.          0.08695652  0.78778135]
 [ 1.          0.91304348  0.21221865]]

Part c:
I implemented the direct computation of P(O|lambda) and the alpha pass as algorithms, exactly like it said, and they worked.

Part d:
For part (a), we have five multiplications for each of the eight possible sequences, so the work factor is 40
In terms of N and T, we have 2^N possible sequences with 2T-1 multiplications each, so work factor is 2^N*(2T-1)
For part (b), we have fourteen multiplications, so the work factor is 14.
In terms of N and T, we have N*(N+1) multiplications per step, except the first step is only N, so total of N+(T-1)*N*(N+1)
We d

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:40: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:43: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Problem 4:

Original: 
$$a_{ij} = \sum_{t=0}^{T-2} \gamma_t(i,j) \bigg/ \sum_{t=0}^{T-2} \gamma_t(i)$$
    
$$b_j(k) = \sum_{t\in {0,1,\dots,T-1}, \mathcal{O}_t=k} \gamma_t(j) \bigg/ \sum_{t=0}^{T-1} \gamma_t(j)$$
                
$$\gamma_t(i,j) = \alpha_t(i)a_{ij}b_j(\mathcal{O}_{t+1})\beta_{t+1}(j)\bigg/ P(\mathcal{O}|\lambda)$$

Therefore, $a_{ij}$ and $b_j(k)$ in terms of $\alpha$ and $\beta$ (since the $P(\mathcal{O}|\lambda)$ cancels) are 

$$a_{ij} = \sum_{t=0}^{T-2} \alpha_t(i)a_{ij}b_j(\mathcal{O}_{t+1})\beta_{t+1}(j)\bigg/ \sum_{t=0}^{T-2} \sum_{j=0}^{N-1} \alpha_t(i)a_{ij}b_j(\mathcal{O}_{t+1})\beta_{t+1}(j)$$

and

$$b_j(k) = \sum_{t\in {0,1,\dots,T-1}, \mathcal{O}_t=k} \sum_{i=0}^{N-1} \alpha_t(j)a_{ji}b_i(\mathcal{O}_{t+1})\beta_{t+1}(i) \bigg/ \sum_{t=0}^{T-1} \sum_{i=0}^{N-1} \alpha_t(j)a_{ji}b_i(\mathcal{O}_{t+1})\beta_{t+1}(i)$$

In [8]:
def beta_pass(A,B,pi,obs,c):
    """
    A,B,pi are the guesses.
    B gives you the betas. B is NxT. so beta_t(i) is the ith row, t-th column
    c is a vector of length T
    """
    T = len(obs) # of observations
    N = A.shape[0] # of states
    M = B.shape[0]
    
    beta = np.zeros_like(B)
    beta[:,-1] = c
    
    for t in xrange(T-2,0,-1):
        for i in xrange(N):
            for j in xrange(N):
                # beta_t(i) += a_ij*b_j(O_t+1)*beta_t+1(j)
                beta[i,t] += A[i,j]*B[obs[t+1],j]*beta[j,t+1]
            beta[i,t] *= c[t]
    return beta
                
def gamma(A,B,alpha,beta):
    # WHAT SHAPE IS GAMMA SUPPOSED TO BE
    
    for t in xrange(T-1):
        denom = 0
        for i in xrange(N):
            for j in xrange(N):
                denom += alpha[i,t]*A[i,j]*B[obs[t+1],j]*beta[j,t+1]
        for i in xrange(N):
            

def reestimate():
    pass


def HMM(A,B,pi):
    """A, B, and pi are the initial guesses."""
    # The alpha pass (return cT)
    
    # The beta pass
    
    # Compute gammas
    
    # Re-estimate
    
    # Compute logprob
    
    # To continue iterating


def prob3(data):
    # Randomly initialize A, B and pi as given
    pi = np.array([0.51316,0.48684])
    A = np.array([[0.47468,0.52532],[0.51656,0.48344]])
    B = 1/27.*np.ones((27,2)) + np.random.normal(loc=0,scale=0.003,size=(27,2))
    
    # Call HMM until done iterating
    # len(obs) = T = 50k
    # N = # of states = 2
    # M = # of observation symbols = 27

# Get the first 50k letters in a nice (' '=26, 'a'=0, etc.) format.
with open('brown.txt','r') as myfile:
    data = myfile.read()
data = data[:50000]
data = list(data)
out = np.zeros(len(data))
for i in xrange(len(data)):
    if ord(data[i]) == 32:
        out[i] = 26
    else:
        out[i] = ord(data[i]) - 97
print 1/27.
prob3(out)

IndentationError: expected an indented block (<ipython-input-8-62da3773c942>, line 29)